In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm, colors
import matplotlib.animation as animation

def create_initial_temperature(room_size, hot_spot, walls):
    temp = np.ones(room_size) * 28  # Base temperature of 28°C
    x, y = np.meshgrid(np.arange(room_size[0]), np.arange(room_size[1]))
    temp += 6 * np.exp(-0.1 * ((x - hot_spot[0])**2 + (y - hot_spot[1])**2))  # Hot spot
    temp[walls] = 28  # Set wall temperature
    return temp

def simulate_temperature_and_airflow(ac_position, ac_direction, ac_strength, fan_position, fan_power, initial_temp, walls, time_steps=10):
    room_size = initial_temp.shape
    temp = initial_temp.copy()
    airflow = np.zeros(room_size + (2,))  # x and y components of airflow
    
    temp_history = [temp.copy()]
    airflow_history = [airflow.copy()]

    for _ in range(time_steps - 1):
        # Calculate Laplacian considering walls (reduced natural heat diffusion)
        laplacian = np.zeros_like(temp)
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            neighbor = np.roll(np.roll(temp, dx, axis=0), dy, axis=1)
            mask = np.roll(np.roll(walls, dx, axis=0), dy, axis=1)
            laplacian += np.where(mask, temp, neighbor) - temp
        
        temp += 0.05 * laplacian  # Reduced natural heat diffusion

        # AC effect (slightly reduced)
        ac_effect = ac_strength * np.exp(-0.05 * np.sqrt(
            (np.arange(room_size[0])[:, None] - ac_position[0])**2 +
            (np.arange(room_size[1])[None, :] - ac_position[1])**2
        ))
        temp -= 0.2 * ac_effect

        # Fan effect (increased)
        fan_effect = fan_power * 0.2 * np.exp(-0.03 * np.sqrt(
            (np.arange(room_size[0])[:, None] - fan_position[0])**2 +
            (np.arange(room_size[1])[None, :] - fan_position[1])**2
        ))
        temp -= 0.2 * fan_effect

        # Update airflow
        airflow *= 0.8  # Increased decay factor
        airflow[ac_position[0], ac_position[1]] += ac_strength * ac_direction
        fan_direction = np.array([-1, 0])  # Fan pulling air towards negative x
        airflow[fan_position[0], fan_position[1]] += fan_power * 0.4 * fan_direction  # Increased fan influence

        # Apply wall effects on airflow
        airflow[walls] = 0

        # Convection (increased effect)
        grad_x, grad_y = np.gradient(temp)
        temp -= 0.2 * (airflow[:,:,0] * grad_x + airflow[:,:,1] * grad_y)

        # Clip temperature to realistic range
        temp = np.clip(temp, 20, 35)

        temp_history.append(temp.copy())
        airflow_history.append(airflow.copy())

    return np.array(temp_history), np.array(airflow_history)

def update_plot(frame, temp_data1, airflow_data1, temp_data2, airflow_data2, X, Y, walls):
    for i, (temp_data, airflow_data, fan_power) in enumerate([(temp_data1, airflow_data1, 0), (temp_data2, airflow_data2, 50)]):
        ax = axes[i]
        ax.clear()
        temp = temp_data[frame]
        airflow = airflow_data[frame]
        temp[walls] = np.nan
        
        # 3D surface plot for temperature
        surf = ax.plot_surface(X, Y, temp, cmap='coolwarm', alpha=0.8, vmin=20, vmax=35, rstride=1, cstride=1,norm=colors.Normalize(vmin=20, vmax=35), edgecolor='none')
        
        # Quiver plot for airflow
        skip = (slice(None, None, 2), slice(None, None, 2))
        quiv = ax.quiver(X[skip], Y[skip], temp[skip], 
                         airflow[skip + (0,)], airflow[skip + (1,)], np.zeros_like(X[skip]),
                         length=0.7, normalize=True, color='black')
        
        # Plot walls (only above z=20)
        wall_x, wall_y = np.where(walls)
        ax.bar3d(wall_x, wall_y - 1, np.full_like(wall_x, 20), 1, 1, 15, color='gray', alpha=0.8)
        
        # Mark AC and fan positions
        ax.scatter(*ac_position, 34, color='blue', s=150, label='AC')
        ax.scatter(*fan_position, 34, color='purple', s=150, label='Fan')
        
        # Add AC direction arrow
        ax.quiver(*ac_position, 34, *ac_direction, 0, color='blue', length=2)
        
        ax.set_xlabel('X', fontsize=16)
        ax.set_ylabel('Y', fontsize=16)
        ax.set_zlabel('Temperature (°C)', fontsize=22)
        title_text = "Fan ON" if fan_power > 0 else "Fan OFF"
        ax.set_title(f'{title_text}', fontsize=26)
        ax.set_zlim(20, 35)
        
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])
    
        ax.grid(True)
        ax.legend(fontsize=14)
    
    return axes

# Simulation parameters
room_size = (20, 20)
hot_spot = (3, 3)  # Location of initial temperature imbalance
ac_position = (0, 18)  # Air conditioner position
ac_direction = np.array([1, 0])  # AC blowing to the right
ac_strength = 2.0  # AC strength
fan_position = (19, 10)  # Fan position
time_steps = 12  # Number of simulation steps

# Define walls (horizontal wall)
walls = np.zeros(room_size, dtype=bool)
walls[0:8, 8] = True  # Horizontal wall in the middle

# Create initial temperature distribution
initial_temp = create_initial_temperature(room_size, hot_spot, walls)

# Create grid
Y, X = np.meshgrid(range(room_size[0]), range(room_size[1]))

# Simulate temperature and airflow for fan power 0% and 100%
temp_data1, airflow_data1 = simulate_temperature_and_airflow(ac_position, ac_direction, ac_strength, fan_position, 0, initial_temp, walls, time_steps)
temp_data2, airflow_data2 = simulate_temperature_and_airflow(ac_position, ac_direction, ac_strength, fan_position, 50, initial_temp, walls, time_steps)

# Set up the figure and 3D axes
fig = plt.figure(figsize=(24, 16))  # Increased overall figure size
axes = [fig.add_subplot(121, projection='3d'), fig.add_subplot(122, projection='3d')]

# Create animation
ani = animation.FuncAnimation(fig, update_plot, frames=len(temp_data1),
                              fargs=(temp_data1, airflow_data1, temp_data2, airflow_data2, X, Y, walls),
                              interval=200, blit=False)

# Add colorbar for temperature
sm = plt.cm.ScalarMappable(cmap='coolwarm', norm=plt.Normalize(vmin=20, vmax=35))
sm.set_array([])
cax = fig.add_axes([0.1, 0.05, 0.8, 0.02])  # [left, bottom, width, height]
cbar = fig.colorbar(sm, cax=cax, orientation='horizontal', aspect=40)
cbar.set_label('Temperature (°C)', fontsize=14)
cbar.ax.tick_params(labelsize=14)

# Adjust layout and add main title
plt.tight_layout()

# Increase font size for axis labels and titles
for ax in axes:
    ax.set_xlabel('X', fontsize=14)
    ax.set_ylabel('Y', fontsize=14)
    ax.set_zlabel('Temperature (°C)', fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.title.set_fontsize(16)
    
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_zticklabels([])
    
    ax.grid(True)

# Save the animation
ani.save(f'room_temp_airflow_simulation_with_wall.gif', writer='pillow', fps=1)
plt.close(fig)

print("Simulation completed and saved as GIF file.")

C:\Users\Lab8\AppData\Local\Temp\ipykernel_23240\2604183846.py:152: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Simulation completed and saved as GIF file.
